In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
#computes the similarity between the enterd value and the valus in our df
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
df = pd.read_csv('skindataall.csv')
df.head()


,Unnamed: 0,Username,Skin_Tone,Skin_Type,Eye_Color,Hair_Color,Rating_Stars,Review,Product,Brand,...,Oily,Sensitive,Category,Product_Url,User_id,Product_id,Ingredients_Cleaned,Review_Cleaned,Good_Stuff,Ing_Tfidf
0,0,allyp3,Medium,Combination,Brown,Brunette,5,This is hands down the best cleanser I’ve ever...,Superfood Antioxidant Cleanser,YOUTH TO THE PEOPLE,...,0,0,Cleanser,https://www.sephora.com/product/kale-spinach-g...,3420,157,"['sodium', 'cocoyl', 'glutamate', 'cocamidopro...","['hand', 'best', 'cleanser', 'ever', 'used', '...",1,"sodium, cocoyl, glutamate, cocamidopropyl, bet..."
1,1,PatTea,Medium,Combination,Brown,Red,1,Unfortunately this doesn’t work for everyone. ...,Superfood Antioxidant Cleanser,YOUTH TO THE PEOPLE,...,0,0,Cleanser,https://www.sephora.com/product/kale-spinach-g...,2483,157,"['sodium', 'cocoyl', 'glutamate', 'cocamidopro...","['unfortunately', 'work', 'everyone', 'used', ...",0,"sodium, cocoyl, glutamate, cocamidopropyl, bet..."
2,2,Sabi1991,No data,No data,No data,No data,5,My favorite cleanser!! i love the packaging on...,Superfood Antioxidant Cleanser,YOUTH TO THE PEOPLE,...,0,0,Cleanser,https://www.sephora.com/product/kale-spinach-g...,2715,157,"['sodium', 'cocoyl', 'glutamate', 'cocamidopro...","['favorite', 'cleanser', 'love', 'packaging', ...",1,"sodium, cocoyl, glutamate, cocamidopropyl, bet..."
3,3,happyface2,Fair,Dry,Blue,Blonde,5,I love all things Youth To The People! This cl...,Superfood Antioxidant Cleanser,YOUTH TO THE PEOPLE,...,0,0,Cleanser,https://www.sephora.com/product/kale-spinach-g...,4497,157,"['sodium', 'cocoyl', 'glutamate', 'cocamidopro...","['love', 'thing', 'youth', 'people', 'cleanser...",1,"sodium, cocoyl, glutamate, cocamidopropyl, bet..."
4,4,kimkix34,Fair,Normal,Green,Blonde,5,I had a trial size of this and was obsessed. M...,Superfood Antioxidant Cleanser,YOUTH TO THE PEOPLE,...,0,0,Cleanser,https://www.sephora.com/product/kale-spinach-g...,5017,157,"['sodium', 'cocoyl', 'glutamate', 'cocamidopro...","['trial', 'size', 'wa', 'obsessed', 'skin', 'f...",1,"sodium, cocoyl, glutamate, cocamidopropyl, bet..."


In [10]:
#removing unwanted col
df.drop('Skin_Tone', inplace=True, axis=1)
df.drop('Eye_Color', inplace=True, axis=1)
df.drop('Hair_Color', inplace=True, axis=1)
df.drop('Product_Url', inplace=True, axis=1)
df.drop('Username', inplace=True, axis=1)
df.drop('Good_Stuff', inplace=True, axis=1)
df.drop('Ingredients', inplace=True, axis=1)
df.drop('Unnamed: 0', inplace=True, axis=1)
df.drop('Review_Cleaned', inplace=True, axis=1)
df.drop('Price', inplace=True, axis=1)
df.drop('User_id', inplace=True, axis=1)
df.drop('Review', inplace=True, axis=1)
df.drop('Skin_Type', inplace=True, axis=1)

In [11]:
#drop the face mask 
df = df[~df.Category.str.match('Face Mask')]

In [12]:
#checking duplicates
df.duplicated(subset='Product_id').sum() 

8311

In [13]:
#removing them
df = df.drop_duplicates(subset='Product_id')

In [14]:
#checking if they were removed
df.duplicated(subset='Product_id').sum() 

0

In [15]:
df.reset_index(drop=True)

,Rating_Stars,Product,Brand,Rating,Combination,Dry,Normal,Oily,Sensitive,Category,Product_id,Ingredients_Cleaned,Ing_Tfidf
0,5,Superfood Antioxidant Cleanser,YOUTH TO THE PEOPLE,4.4,0,0,0,0,0,Cleanser,157,"['sodium', 'cocoyl', 'glutamate', 'cocamidopro...","sodium, cocoyl, glutamate, cocamidopropyl, bet..."
1,5,Cleansing & Exfoliating Wipes,SEPHORA COLLECTION,4.5,0,0,0,0,0,Cleanser,65,"['caprylic', 'capric', 'triglyceride', 'fragra...","caprylic, capric, triglyceride, fragrance, phe..."
2,5,Purity Made Simple Cleanser,PHILOSOPHY,4.5,1,1,1,1,1,Cleanser,210,"['sodium', 'lauroamphoacetate', 'sodium', 'tri...","sodium, lauroamphoacetate, sodium, trideceth, ..."
3,5,Beste™ No. 9 Jelly Cleanser,DRUNK ELEPHANT,4.1,1,0,1,1,1,Cleanser,35,"['sodium', 'lauroyl', 'methyl', 'isethionate',...","sodium, lauroyl, methyl, isethionate, glycerin..."
4,5,The Rice Polish Foaming Enzyme Powder,TATCHA,4.4,1,1,1,1,1,Cleanser,196,"['microcrystalline', 'cellulose', 'oryza', 'sa...","microcrystalline, cellulose, oryza, sativa, ri..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,1,Midnight Secret Late Night Recovery Treatment,GUERLAIN,4.2,1,1,1,1,1,Moisturizer,173,"['visit', 'guerlain', 'boutique']","visit, guerlain, boutique"
310,5,Post-Acne Spot Lightening Gel,MURAD,3.7,0,0,0,0,0,Treatment,198,"['alcohol', 'denat', 'glycolic', 'acid', 'buty...","alcohol, denat, glycolic, acid, butylene, glyc..."
311,2,Hydro-Plumping Re-Texturizing Serum Concentrate,KIEHL'S SINCE 1851,4.7,0,0,0,0,0,Treatment,150,"['glycerin', 'dimethicone', 'propanediol', 'di...","glycerin, dimethicone, propanediol, dimethicon..."
312,5,The Microdelivery Resurfacing Peel,PHILOSOPHY,4.7,0,0,0,0,0,Treatment,266,"['microdelivery', 'peel', 'step', 'peg', 'sodi...","microdelivery, peel, step, peg, sodium, bicarb..."


In [16]:
#clean Ingredients_Cleaned more
df.replace(',','', regex=True, inplace=True)

In [17]:
df.head()

,Rating_Stars,Product,Brand,Rating,Combination,Dry,Normal,Oily,Sensitive,Category,Product_id,Ingredients_Cleaned,Ing_Tfidf
0,5,Superfood Antioxidant Cleanser,YOUTH TO THE PEOPLE,4.4,0,0,0,0,0,Cleanser,157,['sodium' 'cocoyl' 'glutamate' 'cocamidopropyl...,sodium cocoyl glutamate cocamidopropyl betaine...
48,5,Cleansing & Exfoliating Wipes,SEPHORA COLLECTION,4.5,0,0,0,0,0,Cleanser,65,['caprylic' 'capric' 'triglyceride' 'fragrance...,caprylic capric triglyceride fragrance phenoxy...
96,5,Purity Made Simple Cleanser,PHILOSOPHY,4.5,1,1,1,1,1,Cleanser,210,['sodium' 'lauroamphoacetate' 'sodium' 'tridec...,sodium lauroamphoacetate sodium trideceth sulf...
144,5,Beste™ No. 9 Jelly Cleanser,DRUNK ELEPHANT,4.1,1,0,1,1,1,Cleanser,35,['sodium' 'lauroyl' 'methyl' 'isethionate' 'gl...,sodium lauroyl methyl isethionate glycerin coc...
192,5,The Rice Polish Foaming Enzyme Powder,TATCHA,4.4,1,1,1,1,1,Cleanser,196,['microcrystalline' 'cellulose' 'oryza' 'sativ...,microcrystalline cellulose oryza sativa rice p...


In [18]:
#remove zeros and change one to col names in new col 

# changing data types of compatibility columns from int to str so we can manipulate them
df = df.astype({'Combination':'string','Dry':'string','Normal':'string','Oily':'string','Sensitive':'string'})

#replacing all 1's with col name 
# #Combination	Dry	Normal	Oily	Sensitive
df['Combination'] = df['Combination'].str.replace('1','combinational')
df['Combination'] = df['Combination'].str.replace('0',' ')

df['Dry'] = df['Dry'].str.replace('1','dry')
df['Dry'] = df['Dry'].str.replace('0',' ')


df['Normal'] = df['Normal'].str.replace('1','normal')
df['Normal'] = df['Normal'].str.replace('0',' ')


df['Oily'] = df['Oily'].str.replace('1','oily')
df['Oily'] = df['Oily'].str.replace('0',' ')


df['Sensitive'] = df['Sensitive'].str.replace('1','sensitive')
df['Sensitive'] = df['Sensitive'].str.replace('0',' ')

#Adding Skin sutibility column to combine what the product is good for as a feature 
df['Skin_sutibility'] = df['Combination'].map(str) + ' ' + df['Dry'].map(str) + ' ' + df['Normal'].map(str)+ ' ' + df['Oily'].map(str) + ' ' +df['Sensitive'].map(str)

In [19]:
df.head(5)


,Rating_Stars,Product,Brand,Rating,Combination,Dry,Normal,Oily,Sensitive,Category,Product_id,Ingredients_Cleaned,Ing_Tfidf,Skin_sutibility
0,5,Superfood Antioxidant Cleanser,YOUTH TO THE PEOPLE,4.4,,,,,,Cleanser,157,['sodium' 'cocoyl' 'glutamate' 'cocamidopropyl...,sodium cocoyl glutamate cocamidopropyl betaine...,
48,5,Cleansing & Exfoliating Wipes,SEPHORA COLLECTION,4.5,,,,,,Cleanser,65,['caprylic' 'capric' 'triglyceride' 'fragrance...,caprylic capric triglyceride fragrance phenoxy...,
96,5,Purity Made Simple Cleanser,PHILOSOPHY,4.5,combinational,dry,normal,oily,sensitive,Cleanser,210,['sodium' 'lauroamphoacetate' 'sodium' 'tridec...,sodium lauroamphoacetate sodium trideceth sulf...,combinational dry normal oily sensitive
144,5,Beste™ No. 9 Jelly Cleanser,DRUNK ELEPHANT,4.1,combinational,,normal,oily,sensitive,Cleanser,35,['sodium' 'lauroyl' 'methyl' 'isethionate' 'gl...,sodium lauroyl methyl isethionate glycerin coc...,combinational normal oily sensitive
192,5,The Rice Polish Foaming Enzyme Powder,TATCHA,4.4,combinational,dry,normal,oily,sensitive,Cleanser,196,['microcrystalline' 'cellulose' 'oryza' 'sativ...,microcrystalline cellulose oryza sativa rice p...,combinational dry normal oily sensitive


In [20]:
#making the product name in lower case because the user will probably enter it that way
df['Product'] = df['Product'].str.lower()

In [21]:
#now we remove the extra columns does it have to be 
df.drop(['Rating','Combination','Dry','Normal','Oily','Sensitive','Ingredients_Cleaned'],inplace=True,axis=1)

In [22]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 314 entries, 0 to 8626
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Rating_Stars     314 non-null    int64 
 1   Product          314 non-null    object
 2   Brand            314 non-null    object
 3   Category         314 non-null    object
 4   Product_id       314 non-null    int64 
 5   Ing_Tfidf        314 non-null    object
 6   Skin_sutibility  314 non-null    object
dtypes: int64(2), object(5)
memory usage: 19.6+ KB


In [23]:
#checking fo null values
df.isnull().values.any()

False

In [24]:
#changing the index of some columns so that we dont add them in the combined data col
df = df [['Product_id','Product','Rating_Stars','Brand', 'Category','Skin_sutibility','Ing_Tfidf']]

In [25]:
df.sample(5)


,Product_id,Product,Rating_Stars,Brand,Category,Skin_sutibility,Ing_Tfidf
5919,164,martian mattifying melting water-gel toner,1,SUNDAY RILEY,Cleanser,combinational normal oily sensitive,alcohol denat dicaprylyl carbonate dimethyl is...
1512,143,goodnight glow retin-alt sleeping crème,1,OLEHENRIKSEN,Moisturizer,combinational dry normal oily,glycerin vitis vinifera grape seed oil c alkyl...
8580,150,hydro-plumping re-texturizing serum concentrate,2,KIEHL'S SINCE 1851,Treatment,,glycerin dimethicone propanediol dimethicone p...
5776,30,bamboo charcoal detoxifying soap bar,3,HERBIVORE,Cleanser,combinational normal oily,saponified oil coco nucifera coconut oil olea ...
4614,68,cleansing spa water makeup remover,5,KOH GEN DO,Cleanser,,dipropylene glycol peg glyceryl cocoate methyl...


In [26]:
#creating a column with the important features 
df['combined_data'] = df[df.columns[4:]].apply( 
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)

df.sample(10)

,Product_id,Product,Rating_Stars,Brand,Category,Skin_sutibility,Ing_Tfidf,combined_data
4427,54,ceramic slip cleanser,1,SUNDAY RILEY,Cleanser,combinational dry normal oily,disodium laureth sulfosuccinate octyldoecanol ...,Cleanser combinational dry normal oily disod...
4188,279,treatment cleansing foam,5,AMOREPACIFIC,Cleanser,combinational normal oily,glycerin stearic acid myristic acid potassium ...,Cleanser combinational normal oily glyceri...
6965,10,acne healing dots,5,PEACE OUT,Treatment,combinational dry normal oily sensitive,salicylic acid dissolve pore congestion fight ...,Treatment combinational dry normal oily sensit...
4140,66,cleansing wipes - rose - moisturizing,5,SEPHORA COLLECTION,Cleanser,,info,Cleanser info
8064,125,ferulic + retinol wrinkle recovery peel,5,DR. DENNIS GROSS SKINCARE,Treatment,combinational dry normal oily,hamamelis virginiana witch hazel sd alcohol b ...,Treatment combinational dry normal oily hama...
432,32,beauty elixir,5,CAUDALIE,Moisturizer,combinational normal oily sensitive,alcohol citrus aurantium amara bitter orange f...,Moisturizer combinational normal oily sensit...
3473,46,blueberry bounce gentle cleanser,5,GLOW RECIPE,Cleanser,combinational dry normal oily sensitive,palm kernel coco glucoside caulerpa lentillife...,Cleanser combinational dry normal oily sensiti...
6773,303,vitamin c ester brightening serum,3,PERRICONE MD,Treatment,combinational dry normal oily sensitive,tetrahexyldecyl ascorbate caprylyl methicone h...,Treatment combinational dry normal oily sensit...
2780,24,aqua bomb sleeping mask,2,BELIF,Moisturizer,combinational dry normal oily sensitive,dipropylene glycol polyglycerin hexanediol gly...,Moisturizer combinational dry normal oily sens...
4875,100,essential-c cleanser,5,MURAD,Cleanser,,cocamidopropyl betaine disodium laureth sulfos...,Cleanser cocamidopropyl betaine diso...


In [27]:
def recommender (product_name):

#turning our combined data col to a vector to measure its similarity against other vectors
    vec = CountVectorizer()
    vectorized = vec.fit_transform(df['combined_data'])

#calculate cosine sim matrix from the vectors 
    cs = cosine_similarity(vectorized)
#get the name of product 
    #product_name = 'cleanser'
#finding the product id
    product_id = df[df.Product == product_name]['Product_id'].values[0]

#create enumerations for similarity scores and sort it 
    score = list(enumerate(cs[product_id]))
    sorted_score = sorted(score, key = lambda x:x[1], reverse= True)[1:]
    i = 0
    print ('the 5 most similar product to your choice are:\n ')
    for item in sorted_score:

        product_namef = df[df.Product_id == item[0]]['Product'].values[0]
        print(i+1, product_namef , sorted_score[i][1] )
        i = i+1
        if i == 5:
            break
      

In [28]:
recommender('caviar lime acid peel')

the 5 most similar product to your choice are:
 
1 rub-a-dub refining peel gel 0.5413643428790968
2 black tea age-delay firming serum 0.5252548322357349
3 retinol youth renewal serum 0.5223564126077063
4 ceramidin™ cream 0.5157106231293968
5 plantscription™ anti-aging cleanser 0.5088344814876714


In [29]:
recommender('advanced acne & wrinkle reducer')

the 5 most similar product to your choice are:
 
1 pore-balance™ facial sauna scrub 0.5304366247201397
2 advanced active radiance® serum 0.4653959766335518
3 the rice polish foaming enzyme powder 0.4562667580712162
4 protein booster skin serum 0.45289472989165636
5 ultra repair® cream intense hydration 0.45198461470556855


In [30]:
recommender('treatment toner')

the 5 most similar product to your choice are:
 
1 ultra facial toner 0.6403607079360155
2 ultra facial oil-free gel-cream 0.6334971259181914
3 deep cleansing exfoliator 0.6303375195568406
4 supercleanse™ clearing cream-to-foam cleanser 0.6302148639389665
5 anti bac clearing lotion 0.6289780933540263


In [31]:
recommender('liquid facial soap')

the 5 most similar product to your choice are:
 
1 ceramic slip cleanser 0.6807792619171564
2 peat miracle revital cream 0.6792835272577389
3 100 percent pure argan oil 0.6790130277306338
4 crème radiance gentle cleansing creamy-foam cleanser 0.6708203932499367
5 essential power skin toner for combination to oily skin 0.6608271381079533


In [32]:
recommender('acne healing dots')

the 5 most similar product to your choice are:
 
1 facial treatment cleanser 0.9999999999999999
2 perfect cleansing oil 0.9999999999999999
3 juno antioxidant + superfood face oil 0.6708203932499369
4 plantscription™ spf 25 power anti-aging cream 0.6708203932499369
5 prep-n-glow™ cloths 0.6708203932499369


In [33]:
recommender('beauty elixir')

the 5 most similar product to your choice are:
 
1 black tea age-delay cream 0.6235100623532127
2 blueberry bounce gentle cleanser 0.5281643011010203
3 purity made simple cleanser 0.5046560756374135
4 instant long-wear makeup remover 0.4594594594594595
5 evercalm™ anti-redness serum 0.4585350281266174
